<a href="https://colab.research.google.com/github/mhrhpr/civil_daily_report/blob/main/civil_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("/content/orbital-avatar-467611-c6-66d2b475fbf6.json", scope)
client = gspread.authorize(creds)

sheet = client.open("گزارش روزانه کارگاه").worksheet("sheet1")


In [3]:
!pip install python-telegram-bot==22.3 gspread google-auth nest_asyncio




In [4]:
!pip install python-telegram-bot==22.3


In [ ]:
import nest_asyncio
import gspread
from google.oauth2.service_account import Credentials
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Application, CommandHandler, CallbackQueryHandler, MessageHandler, ContextTypes, filters

nest_asyncio.apply()

# Google Sheets credentials
creds = Credentials.from_service_account_file(
    '/content/orbital-avatar-467611-c6-66d2b475fbf6.json',
    scopes=['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
)
gc = gspread.authorize(creds)
spreadsheet = gc.open('گزارش روزانه کارگاه')
worksheet = spreadsheet.sheet1

# Ensure headers
if worksheet.row_count == 0 or not worksheet.row_values(1):
    worksheet.append_row(['chat_id', 'نوع', 'نام', 'زمینه', 'واحد', 'مزد مبنا', 'تعداد', 'مبلغ کل', 'نوع اضافه‌کاری', 'مبلغ اضافه‌کار'])

TOKEN = "8288041222:AAE2MaWaW8tliGSxOi9yfeBFzBLL1Nx2Tnk"
CHAT_ID = 380490903
user_states = {}

def get_keyboard(options):
    return InlineKeyboardMarkup.from_column([InlineKeyboardButton(opt, callback_data=opt) for opt in options])

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    keyboard = get_keyboard(['شروع 📋'])
    await update.message.reply_text("سلام 👋 به ربات گزارش روزانه خوش اومدی.\nبرای شروع دکمه زیر رو بزن:", reply_markup=keyboard)
    user_states[update.effective_chat.id] = {'step': 'waiting_start'}

async def button_handler(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    await query.answer()
    chat_id = query.message.chat_id
    text = query.data
    state = user_states.get(chat_id, {})

    if state.get('step') == 'waiting_start' and text == 'شروع 📋':
        keyboard = get_keyboard(['بله', 'خیر'])
        await query.edit_message_text("آیا امشب گزارشی داری؟", reply_markup=keyboard)
        user_states[chat_id]['step'] = 'start'

    elif state.get('step') == 'start':
        if text == 'خیر':
            await query.edit_message_text("باشه، فردا می‌بینمت 🌙")
            user_states.pop(chat_id, None)
        elif text == 'بله':
            user_states[chat_id] = {'step': 'type'}
            await query.edit_message_text("گزارش مربوط به چه کسی است؟")
            keyboard = get_keyboard(['پیمانکار', 'روزمزد'])
            await context.bot.send_message(chat_id=chat_id, text="پیمانکار یا روزمزد؟", reply_markup=keyboard)

    elif state.get('step') == 'type':
        state['person_type'] = text
        state['step'] = 'ask_name'
        await query.edit_message_text(f"نام {text} را وارد کن:")

    elif state.get('step') == 'ask_unit_choice':
        state['unit'] = text
        state['step'] = 'ask_base'
        await query.edit_message_text("مزد مبنا را وارد کن:")

    elif state.get('step') == 'ask_ot_type':
        if text in ['قرارداد', 'ندارد']:
            state['ot_type'] = text
            state['ot_amount'] = '0'
            state['step'] = 'finalize'
            await finalize_report(chat_id, context)
        else:
            state['ot_type'] = 'اضافه‌کار'
            state['step'] = 'ask_ot_amount'
            await query.edit_message_text("مبلغ اضافه‌کار را وارد کن:")

    elif state.get('step') == 'post_register':
        if text == 'پایان گزارش':
            await query.edit_message_text("موفق باشی تا بعد 👋")
            user_states.pop(chat_id, None)
        else:
            user_states[chat_id] = {'step': 'type'}
            await query.edit_message_text("گزارش جدید: پیمانکار یا روزمزد؟")
            keyboard = get_keyboard(['پیمانکار', 'روزمزد'])
            await context.bot.send_message(chat_id=chat_id, text="نوع گزارش را انتخاب کن:", reply_markup=keyboard)

async def message_handler(update: Update, context: ContextTypes.DEFAULT_TYPE):
    chat_id = update.effective_chat.id
    text = update.message.text.strip()
    state = user_states.get(chat_id, {})

    if state.get('step') == 'ask_name':
        state['name'] = text
        state['step'] = 'ask_field'
        await update.message.reply_text("زمینه کاری چیست؟" if state['person_type'] == 'پیمانکار' else "زمینه شغلی چیست؟")

    elif state.get('step') == 'ask_field':
        state['field'] = text
        if state['person_type'] == 'پیمانکار':
            state['step'] = 'ask_unit_choice'
            keyboard = get_keyboard(['متر', 'متر مربع', 'واحد'])
            await update.message.reply_text("واحد اندازه‌گیری را انتخاب کن:", reply_markup=keyboard)
        else:
            state['step'] = 'ask_unit'
            await update.message.reply_text("واحد را وارد کن:")

    elif state.get('step') == 'ask_unit':
        state['unit'] = text
        state['step'] = 'ask_base'
        await update.message.reply_text("مزد مبنا را وارد کن:")

    elif state.get('step') == 'ask_base':
        state['base'] = text
        state['step'] = 'ask_count'
        await update.message.reply_text("تعداد را وارد کن:")

    elif state.get('step') == 'ask_count':
        state['count'] = text
        state['step'] = 'ask_ot_type'
        if state['person_type'] == 'پیمانکار':
            keyboard = get_keyboard(['قرارداد', 'اضافه‌کار'])
            await update.message.reply_text("نوع را انتخاب کن:", reply_markup=keyboard)
        else:
            keyboard = get_keyboard(['بله', 'ندارد'])
            await update.message.reply_text("اضافه‌کاری داشته؟", reply_markup=keyboard)

    elif state.get('step') == 'ask_ot_amount':
        state['ot_amount'] = text
        state['step'] = 'finalize'
        await finalize_report(chat_id, context)

async def finalize_report(chat_id, context):
    state = user_states.get(chat_id, {})
    try:
        total = float(state['base']) * float(state['count']) + float(state.get('ot_amount', 0))
        worksheet.append_row([
            chat_id,
            state.get('person_type', ''),
            state.get('name', ''),
            state.get('field', ''),
            state.get('unit', ''),
            state.get('base', ''),
            state.get('count', ''),
            str(total),
            state.get('ot_type', ''),
            state.get('ot_amount', '')
        ])
        await context.bot.send_message(chat_id=chat_id, text="✅ گزارش با موفقیت ثبت شد.")
    except Exception as e:
        await context.bot.send_message(chat_id=chat_id, text="❌ متاسفانه ثبت نشد. لطفا دوباره امتحان کن.")
        user_states[chat_id] = {'step': 'type'}
        keyboard = get_keyboard(['پیمانکار', 'روزمزد'])
        await context.bot.send_message(chat_id=chat_id, text="نوع گزارش را انتخاب کن:", reply_markup=keyboard)
        return

    user_states[chat_id]['step'] = 'post_register'
    keyboard = get_keyboard(['ثبت مجدد گزارش', 'پایان گزارش'])
    await context.bot.send_message(chat_id=chat_id, text="آیا مایلی گزارش دیگری ثبت کنی؟", reply_markup=keyboard)

# اجرای ربات
app = Application.builder().token(TOKEN).build()
app.add_handler(CommandHandler("start", start))
app.add_handler(CallbackQueryHandler(button_handler))
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, message_handler))

app.run_polling()
